<a href="https://colab.research.google.com/github/ayegalapati/RoomMe/blob/main/RoomMe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flask_sqlalchemy firebase-admin scikit-learn Flask flask_cors

In [2]:
# Import required libraries
from flask import Flask, request, jsonify
from flask_sqlalchemy import SQLAlchemy
from firebase_admin import auth, initialize_app, credentials
from flask_cors import CORS
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import joblib
import os

In [3]:
# Initialize Flask app
app = Flask(__name__)
CORS(app)  # Enable CORS for cross-origin requests
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///roomme_db.db'  # Use SQLite for Colab simplicity
db = SQLAlchemy(app)


In [5]:
# Initialize Firebase (Check if credential file exists, else prompt to upload)
if os.path.exists('/content/firebase_cred.json'):
    cred = credentials.Certificate('/content/firebase_cred.json')
    initialize_app(cred)
else:
    raise FileNotFoundError("Firebase credential file not found. Please upload firebase_cred.json to Colab.")

In [6]:
# Database Models
class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    email = db.Column(db.String(100), unique=True, nullable=False)
    location = db.Column(db.String(100))
    age = db.Column(db.Integer)
    gender = db.Column(db.String(50))
    rent = db.Column(db.Float)
    looking_for = db.Column(db.String(100))  # Room or Roommate
    occupation = db.Column(db.String(100))
    smoking_preference = db.Column(db.String(50))
    cleanliness = db.Column(db.String(50))
    sleeping_hours = db.Column(db.String(100))
    guest_policy = db.Column(db.String(100))
    pet_policy = db.Column(db.String(100))
    preferences = db.Column(db.JSON)

class Match(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    user1_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    user2_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    success = db.Column(db.Boolean, default=False)

In [7]:
# Seed Initial Data (Optional)
def seed_data():
    if User.query.count() == 0:
        data = pd.read_csv('/content/mock_data.csv')
        for _, row in data.iterrows():
            user = User(
                email=row['email'],
                location=row['location'],
                age=row['age'],
                gender=row['gender'],
                rent=row['rent'],
                looking_for=row['looking_for'],
                occupation=row['occupation'],
                smoking_preference=row['smoking_preference'],
                cleanliness=row['cleanliness'],
                sleeping_hours=row['sleeping_hours'],
                guest_policy=row['guest_policy'],
                pet_policy=row['pet_policy'],
                preferences={}
            )
            db.session.add(user)
        db.session.commit()


In [8]:

# Train Model (Initial Training with Mock Data)
def train_model():
    data = pd.read_csv('/content/mock_data.csv')  # Ensure mock_data.csv is uploaded to Colab
    X = data[['age', 'rent']]
    y = data['match_success']
    model = RandomForestClassifier()
    model.fit(X, y)
    joblib.dump(model, '/content/match_model.pkl')

In [9]:
if 'signup' in app.view_functions:
    app.view_functions.pop('signup')

@app.route('/signup', methods=['POST'])
def signup():
    data = request.json
    email = data['email']
    password = data['password']

    try:
        user = auth.create_user(email=email, password=password)
        new_user = User(email=email)
        db.session.add(new_user)
        db.session.commit()
        return jsonify({"message": "User created", "uid": user.uid}), 201
    except Exception as e:
        return jsonify({"error": str(e)}), 400

In [10]:
if 'create_profile' in app.view_functions:
    app.view_functions.pop('create_profile')

@app.route('/profile', methods=['POST'])
def create_profile():
    data = request.json
    user = User.query.filter_by(email=data['email']).first()
    if not user:
        return jsonify({"error": "User not found"}), 404

    user.location = data['location']
    user.age = data['age']
    user.gender = data['gender']
    user.rent = data['rent']
    user.looking_for = data['looking_for']
    user.occupation = data['occupation']
    user.smoking_preference = data['smoking_preference']
    user.cleanliness = data['cleanliness']
    user.sleeping_hours = data['sleeping_hours']
    user.guest_policy = data['guest_policy']
    user.pet_policy = data['pet_policy']
    user.preferences = data['preferences']

    db.session.commit()
    return jsonify({"message": "Profile updated"}), 200

In [11]:
if 'predict_match' in app.view_functions:
    app.view_functions.pop('predict_match')

@app.route('/predict_match', methods=['POST'])
def predict_match():
    data = request.json
    model = joblib.load('/content/match_model.pkl')
    X = pd.DataFrame([[data['age'], data['rent']]], columns=['age', 'rent'])
    prediction = model.predict(X)

    return jsonify({"match_likelihood": prediction[0]}), 200

In [12]:
if 'start_chat' in app.view_functions:
    app.view_functions.pop('start_chat')

@app.route('/chat', methods=['POST'])
def start_chat():
    data = request.json
    user1 = data['user1_id']
    user2 = data['user2_id']
    match = Match(user1_id=user1, user2_id=user2)
    db.session.add(match)
    db.session.commit()
    return jsonify({"message": "Chat started!"}), 201

In [13]:
import os
print(os.path.exists('/content/mock_data.csv'))  # Should return True

True


In [14]:
import pandas as pd

data = pd.read_csv('/content/mock_data.csv')
print(data.head())  # Show the first few rows
print(data.columns)  # List all column names

Empty DataFrame
Columns: [email, location, age, gender, rent, looking_for, occupation, smoking_preference, cleanliness, sleeping_hours, guest_policy, pet_policy, match_success user1@example.com, New York, 25, Female, 1200, Roommate, Engineer, No, Clean, 10pm-6am, Occasionally, No pets, 1 user2@example.com, Los Angeles, 30, Male, 1500, Room, Designer, Yes, Moderate, 12am-8am, Frequently, Has pets, 0 user3@example.com, San Francisco, 28, Female.1, 1800, Roommate.1, Artist, No.1, Very Clean, 9pm-5am, Rarely, No pets.1, 1 user4@example.com, Chicago, 35, Male.1, 900, Room.1, Teacher, Yes.1, Clean.1, 11pm-7am, Never, Has pets.1, 0 user5@example.com, Austin, 22, Male.2, 800, Roommate.2, Student, No.2, Moderate.1, 1am-9am, Occasionally.1, No pets.2, 1 user6@example.com, Miami, 29, Female.2, 1400, Room.2, Doctor, No.3, Very Clean.1, 10pm-6am.1, Frequently.1, No pets.3, 1 user7@example.com, Seattle, 31, Other, 1300, Roommate.3, Engineer.1, Yes.2, Clean.2, 12am-8am.1, Rarely.1, Has pets.2, 0 user

In [15]:
def seed_data():
    mock_users = [
        {'email': 'user1@example.com', 'location': 'New York', 'age': 25, 'gender': 'Female', 'rent': 1200,
         'looking_for': 'Roommate', 'occupation': 'Engineer', 'smoking_preference': 'No',
         'cleanliness': 'Clean', 'sleeping_hours': '10pm-6am', 'guest_policy': 'Occasionally',
         'pet_policy': 'No pets', 'preferences': {}},

        {'email': 'user2@example.com', 'location': 'Los Angeles', 'age': 30, 'gender': 'Male', 'rent': 1500,
         'looking_for': 'Room', 'occupation': 'Designer', 'smoking_preference': 'Yes',
         'cleanliness': 'Moderate', 'sleeping_hours': '12am-8am', 'guest_policy': 'Frequently',
         'pet_policy': 'Has pets', 'preferences': {}},

        {'email': 'user3@example.com', 'location': 'Chicago', 'age': 28, 'gender': 'Non-binary', 'rent': 1000,
         'looking_for': 'Roommate', 'occupation': 'Teacher', 'smoking_preference': 'No',
         'cleanliness': 'Clean', 'sleeping_hours': '9pm-5am', 'guest_policy': 'Rarely',
         'pet_policy': 'No pets', 'preferences': {}}
    ]

    for user_data in mock_users:
        existing_user = User.query.filter_by(email=user_data['email']).first()
        if not existing_user:
            new_user = User(**user_data)
            db.session.add(new_user)

    db.session.commit()
    print("Database seeded successfully.")

In [16]:
def train_model():
    data = pd.read_csv('/content/mock_data.csv')

    # Handle missing 'match_success' column
    if 'match_success' not in data.columns:
        print("Column 'match_success' not found. Generating random values.")
        import numpy as np
        data['match_success'] = np.random.randint(0, 2, size=len(data))  # Random 0 or 1

    X = data[['age', 'rent']]
    y = data['match_success']

    model = RandomForestClassifier()
    model.fit(X, y)
    joblib.dump(model, '/content/match_model.pkl')
    print("Model trained successfully.")

In [18]:
def train_model():
    data = pd.read_csv('/content/mock_data.csv')

    # Handle missing 'match_success' column
    if 'match_success' not in data.columns:
        print("Column 'match_success' not found. Generating random values.")
        import numpy as np
        data['match_success'] = np.random.randint(0, 2, size=len(data))  # Random 0 or 1

    # Check if DataFrame is empty after potentially adding 'match_success'
    if data.empty:
        print("DataFrame is empty. Cannot train the model.")
        return  # Exit the function early to avoid the error

    X = data[['age', 'rent']]
    y = data['match_success']

    model = RandomForestClassifier()
    model.fit(X, y)
    joblib.dump(model, '/content/match_model.pkl')
    print("Model trained successfully.")

In [23]:

if 'feedback' in app.view_functions:
    app.view_functions.pop('feedback')

@app.route('/feedback', methods=['POST'])
def feedback():
    data = request.json
    match = Match.query.get(data['match_id'])
    match.success = data['success']
    db.session.commit()
    return jsonify({"message": "Feedback received"}), 200

if __name__ == '__main__':
    with app.app_context():
        db.create_all()  # Create tables in SQLite
        seed_data()  # Seed initial data from CSV
    if not os.path.exists('/content/match_model.pkl'):
        train_model()
    app.run(host='0.0.0.0', port=5000)

Database seeded successfully.
Column 'match_success' not found. Generating random values.
DataFrame is empty. Cannot train the model.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
